In [2]:
import pandas as pd
import numpy as np
import os
import geopandas as gp

In [3]:
# -- read the inspection files
try:
    inspection
except:
    print("reading PIP_InspectionMain.xlsx...")
    in_path = os.path.join('/projects/cusp/10266/0/quality_assessment','PIP')
    in_name = os.path.join(in_path,'PIP_InspectionMain.xlsx')
    inspection  = pd.read_excel(in_name)

reading PIP_InspectionMain.xlsx...


In [4]:
# -- get all the ratings information
try:
    rating
except:
    print("reading PIP_ALLSITES.xlsx...")
    as_path  = os.path.join('/projects/cusp/10266/0/quality_assessment','PIP')
    as_name  = os.path.join(as_path,'PIP_FeatureRatings.xlsx')
    rating = pd.read_excel(as_name)
    
    # -- Sustitute numerical values to ratings
    rating.loc[rating['Rating'] == 'a', ['Rating']] = 1
    rating.loc[rating['Rating'] == 'A', ['Rating']] = 1
    rating.loc[rating['Rating'] == 'N', ['Rating']] = np.nan
    rating.loc[rating['Rating'] == 'U', ['Rating']] = 0
    rating.loc[rating['Rating'] == 'U/S', ['Rating']] = 0

reading PIP_ALLSITES.xlsx...


In [5]:
# -- get all the sites information
try:
    sites
except:
    print("reading PIP_ALLSITES.xlsx...")
    as_path  = os.path.join('/projects/cusp/10266/0/quality_assessment','PIP')
    as_name  = os.path.join(as_path,'PIP_ALLSITES.xlsx')
    sites = pd.read_excel(as_name)

reading PIP_ALLSITES.xlsx...


In [6]:
# -- read in the property files
try:
    prop
except:
    print("reading Property.shp...")
    pr_path = os.path.join('/scratch/share/gdobler/parqa/output','CUSPExportShps')
    pr_name = os.path.join(pr_path,'Property.shp')
    prop    = gp.GeoDataFrame.from_file(pr_name)

reading Property.shp...


In [7]:
# -- pull off only first zip in list (if there are multiple)
prop.ZIPCODE = prop.ZIPCODE.apply(lambda x: x[:5])

In [8]:
# -- Change Rating type to float for calculations
rating[['Rating']] = rating[['Rating']].astype(float)

In [9]:
# -- Create pivot table of inspections.
pivotInspect = rating.pivot(index='Inspection ID', columns = 'Feature', values = 'Rating')

In [10]:
# -- create the base ID column  MAY HAVE TO BE TWEAKED
inspection['PID_base'] = [i.split('-')[0].replace('Z','') 
                      for i in inspection['Prop ID']]

In [11]:
# -- Remove Greenstreets from sites
sites = sites[sites['Category'] != 'Greenstreet']

In [12]:
# -- Merge inspection report with sites [Greenstreets are removed from sites]
finalDF = pd.merge(sites, inspection, on = 'Prop ID', how = 'left')

In [13]:
# -- Drop sites that have inspections
finalDF = finalDF[finalDF['Inspection ID'].notnull()]

In [14]:
# -- Merge frame with pivot table of inspections on Inspection ID
finalDF = pd.merge(finalDF, pivotInspect, left_on='Inspection ID', right_index = True, how='left')

In [15]:
##### -- Merge the inspection and properties information
finalDF = pd.merge(finalDF, prop, 'left', left_on='PID_base', 
                  right_on='GISPROPNUM')

In [16]:
# -- check if there are inspected parks that aren't in allsites
check_ALLSITES = False
if check_ALLSITES:
    print("checking allsites file...")
    for ii,pid in enumerate(inspection['Prop ID']):
        flag = True
        for asid in sites['Prop ID']:
            if pid==asid:
                flag = False
                break
        if flag:
            print("couldn't find {0} : {1}".format(ii,pid))

In [17]:
# -- check features file
check_FEATRAT = False
if check_FEATRAT:
    for ii,iid in enumerate(inspection['Inspection ID']):
        if iid not in pivotInspect.index:
            print("couldn't find {0} : Inspection ID: {1}".format(ii,iid))

In [18]:
# -- check properties
check_PROP = False
if check_PROP:
    print("checking properties file...")
    pid_bad = []
    pid_bad_ii = []
    for ii,pid in enumerate(inspection['PID_base']):
        flag = True
        for gpn in prop.GISPROPNUM:
            if pid==gpn:
                flag = False
                break
        if flag:
            if pid not in pid_bad:
                pid_bad_ii.append(ii)
                pid_bad.append(pid)
            print("couldn't find {0} : {1}".format(ii,pid))

    for ii in range(len(pid_bad_ii)):
        subcat = sites.iloc[sites[sites['Prop ID'] == \
                                inspection.iloc[pid_bad_ii[ii]]['Prop ID']] \
                                .index[0]]['Sub-Category']
        print("{0:8} : {1}" \
                  .format(inspection.iloc[pid_bad_ii[ii]]['Prop ID'],subcat))

In [19]:
# -- export Year files
#out_year_path = os.path.join('../../parqa/Outputs','Dataframes')
out_year_path = os.path.join('/scratch/share/gdobler/parqa/output','Dataframes')
inspecYears = sorted(set(map(lambda x: x.year, finalDF['Date'])))

for year in inspecYears:
    yearDF = finalDF[finalDF['Date'].map(lambda x: x.year) == year].reset_index(drop=True)
    out_year_filename = os.path.join(out_year_path, str(year) + '_Inspections.csv')
    yearDF.to_csv(out_year_filename, sep='\t', encoding='utf-8')

IOError: [Errno 2] No such file or directory: '/scratch/share/gdobler/parqa/output/Dataframes/2004_Inspections.csv'

In [20]:
# -- test Read
#yearDemoDF = pd.DataFrame.from_csv(out_year_filename.replace('15', '08'), sep='\t')
yearDemoDF = finalDF

In [21]:
# -- categories
categories = [u'Athletic Fields', u'Benches',
       u'Fences', u'Glass', u'Graffiti', u'Horticultural Areas', u'Ice',
       u'Lawns', u'Litter', u'Paved Surfaces', u'Play Equipment',
       u'Safety Surface', u'Sidewalks', u'Trails', u'Trees', u'Water Bodies',
       u'Weeds']

In [22]:
# -- calculate inspection scores [ratio]
yearDemoDF['Ratings Ratio'] = yearDemoDF[categories].mean(axis=1)

In [23]:
# -- seed final park stats DF
byParkID = yearDemoDF[['Prop ID', 'PID_base', 'Category', 'ZIPCODE']].groupby(['Prop ID'], as_index = False)
parkStats = byParkID.first()

# -- average Park Score and Acres
byParkBase = yearDemoDF.groupby(['PID_base'], as_index = False)
parkAvgScore = byParkBase['Ratings Ratio','ACRES_x'].mean()
parkAvgScore.rename(columns={'Ratings Ratio':'Avg Ratio'}, inplace=True)
parkStats = pd.merge(parkStats, parkAvgScore, on='PID_base', how='left')

# -- calculate weighted Score for park
parkStats['Weighted Score'] = parkStats['Avg Ratio'] * parkStats['ACRES_x']

In [24]:
# -- seed district_category stats DF
byCategory = parkStats.groupby(['ZIPCODE', 'Category'], as_index = False)
categoryStats = byCategory.first()[['ZIPCODE', 'Category']]

# -- sum Acres and Scores per category
catSums = byCategory['ACRES_x', 'Weighted Score'].sum()
catSums.rename(columns={'Weighted Score':'Score'}, inplace=True)
categoryStats = pd.merge(categoryStats, catSums, on=['ZIPCODE', 'Category'], how='left')

# -- count parks per category
catCounts = byCategory.size().reset_index().rename(columns={0:'Counts'})
categoryStats = pd.merge(categoryStats, catCounts, on=['ZIPCODE', 'Category'], how='left')

In [25]:
# -- group by zipcode and category
byZIPCODE = categoryStats.groupby('ZIPCODE', as_index = False)
ZIPCODESums = byZIPCODE.sum()[['ZIPCODE', 'Counts']]

categoryStats = pd.merge(categoryStats, ZIPCODESums, on='ZIPCODE', how = 'left', suffixes = ['_Cat', '_ZIPCODE'])

In [26]:
# -- calculate weighted score for zipcode
categoryStats['Weighted Score'] = categoryStats['Score'] / categoryStats['ACRES_x']
categoryStats['Normalized Score'] = categoryStats['Weighted Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_ZIPCODE'])

In [27]:
# -- sum Scores per Zipcode
byZIPCODE = categoryStats.groupby('ZIPCODE')
ZIPCODEStats = byZIPCODE \
                    .sum() \
                    .reset_index() \
                    [['ZIPCODE', 'Normalized Score']]

In [28]:
#Write to csv
ZIPCODEStats.to_csv('ParkQualityZipcode.csv', sep=',')

In [30]:
pd.read_csv('ParkQualityZipcode.csv')

,Unnamed: 0,ZIPCODE,Normalized Score
0,0,10001,0.955859
1,1,10002,0.863230
2,2,10003,0.927281
3,3,10004,0.936243
4,4,10005,0.983889
5,5,10007,0.973775
6,6,10009,0.907813
7,7,10010,0.953302
8,8,10011,0.946098
9,9,10012,0.937661
